In [45]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import torch
from torch import optim
from torch.autograd import Variable as V
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score

player_stat_00_01 = pd.read_csv("play_per_game_stat/00-01.csv",delimiter = ",")        
player_stat_01_02 = pd.read_csv("play_per_game_stat/01-02.csv",delimiter = ",")
player_stat_02_03 = pd.read_csv("play_per_game_stat/02-03.csv",delimiter = ",")
player_stat_03_04 = pd.read_csv("play_per_game_stat/03-04.csv",delimiter = ",")
player_stat_04_05 = pd.read_csv("play_per_game_stat/04-05.csv",delimiter = ",")
player_stat_05_06 = pd.read_csv("play_per_game_stat/05-06.csv",delimiter = ",")
player_stat_06_07 = pd.read_csv("play_per_game_stat/06-07.csv",delimiter = ",")
player_stat_07_08 = pd.read_csv("play_per_game_stat/07-08.csv",delimiter = ",")
player_stat_08_09 = pd.read_csv("play_per_game_stat/08-09.csv",delimiter = ",")
player_stat_09_10 = pd.read_csv("play_per_game_stat/09-10.csv",delimiter = ",")

player_stat_10_11 = pd.read_csv("play_per_game_stat/10-11.csv",delimiter = ",")
player_stat_11_12 = pd.read_csv("play_per_game_stat/11-12.csv",delimiter = ",")
player_stat_12_13 = pd.read_csv("play_per_game_stat/12-13.csv",delimiter = ",")
player_stat_13_14 = pd.read_csv("play_per_game_stat/13-14.csv",delimiter = ",")
player_stat_14_15 = pd.read_csv("play_per_game_stat/14-15.csv",delimiter = ",")
player_stat_15_16 = pd.read_csv("play_per_game_stat/15-16.csv",delimiter = ",")
player_stat_16_17 = pd.read_csv("play_per_game_stat/16-17.csv",delimiter = ",")

stat_00_10 = [player_stat_00_01,player_stat_01_02,player_stat_02_03,player_stat_03_04,player_stat_04_05,
             player_stat_05_06,player_stat_06_07,player_stat_07_08,player_stat_08_09,player_stat_09_10]
              
stat_10_17 = [ player_stat_10_11,player_stat_11_12,player_stat_12_13,
              player_stat_13_14,player_stat_14_15,player_stat_15_16,player_stat_16_17] 

def initialize_mvp_data(season_begin, season_end):
    a = []
    for i in np.arange(season_begin,season_end,1):
        mvp_dataframe = pd.read_csv("mvp/"+str(i)+"_"+str(i+1)+"_mvp_info.csv",delimiter = ",")
        mvp_dataframe.fillna(0,inplace = True)
        column_num = mvp_dataframe.shape[0]
        for j in range(column_num):
            if(mvp_dataframe.loc[j,"Player"].find("*") != -1):
                mvp_dataframe.loc[j,"Player"] = mvp_dataframe.loc[j,"Player"][0:mvp_dataframe.loc[j,"Player"].find("*")]
            if(mvp_dataframe.loc[j,"Player"].find("\\") != -1):
                mvp_dataframe.loc[j,"Player"] = mvp_dataframe.loc[j,"Player"][0:mvp_dataframe.loc[j,"Player"].find("\\")] 
        a.append(mvp_dataframe)
    return a

mvp_train_stat = initialize_mvp_data(0,10)
mvp_test_stat = initialize_mvp_data(10,17)
print(len(mvp_test_stat))

def initialize_data(stat,mvp_stat_each_season):
    a=[]
    mvp_list = []
    dataframe = pd.read_csv("play_per_game_stat/template.csv",delimiter = ",")
    dataframe = dataframe.drop(["Pos","Tm","Rk"],axis=1)
    for i in range(len(stat)):
        
        print("*"*100)
        print(i)
        a.append(stat[i].drop(["Pos","Tm","Rk"],axis=1))
        column_num = a[i].shape[0]
        a[i].fillna(0,inplace = True)
        for j in range(column_num):
            if(a[i].loc[j,"Player"].find("*") != -1):
                a[i].loc[j,"Player"] = a[i].loc[j,"Player"][0:a[i].loc[j,"Player"].find("*")]
            if(a[i].loc[j,"Player"].find("\\") != -1):
                a[i].loc[j,"Player"] = a[i].loc[j,"Player"][0:a[i].loc[j,"Player"].find("\\")]
            flag = True
            for w in range(mvp_stat_each_season[i].shape[0]):
                if (a[i].loc[j,"Player"] == mvp_stat_each_season[i].loc[w,"Player"]):
                    print(a[i].loc[j,"Player"])
                    mvp_list.append(mvp_stat_each_season[i].loc[w,"Share"])
                    flag = False
                    break
            if(flag):
                mvp_list.append(0)
        dataframe = dataframe.append(a[i])
        dataframe = dataframe.reset_index(drop=True)  # 来改变index就可以了,
    dataframe_with_player_name = dataframe
    dataframe = dataframe.drop(["Player"],axis=1)
    dataframe = pd.DataFrame(dataframe,dtype=np.float)
    return (dataframe_with_player_name,dataframe,mvp_list)

dataframe_00_10_with_player_name, dataframe_00_10 , mvp_list_train  = initialize_data(stat_00_10,mvp_train_stat)
print("|"*50)
dataframe_10_17_with_player_name ,dataframe_10_17, mvp_list_test = initialize_data(stat_10_17,mvp_test_stat)


print(len(mvp_list_test))

print("10-17 mvps:")
for i in range(1623):
    if(mvp_list_test[i] == 0.004):
        print(dataframe_10_17_with_player_name.loc[i,"Player"] , i)
        

7
****************************************************************************************************
0
Allen Iverson
Jerry Stackhouse
Shaquille O'Neal
Kobe Bryant
Vince Carter
Chris Webber
Tracy McGrady
Paul Pierce
Karl Malone
Tim Duncan
Ray Allen
Kevin Garnett
Michael Finley
Jason Kidd
Anthony Mason
David Robinson
John Stockton
****************************************************************************************************
1
Allen Iverson
Shaquille O'Neal
Paul Pierce
Tracy McGrady
Tim Duncan
Kobe Bryant
Dirk Nowitzki
Gary Payton
Jerry Stackhouse
Kevin Garnett
Peja Stojaković
Elton Brand
Steve Nash
Jason Kidd
Mike Bibby
Ben Wallace
****************************************************************************************************
2
Tracy McGrady
Kobe Bryant
Allen Iverson
Shaquille O'Neal
Paul Pierce
Dirk Nowitzki
Tim Duncan
Kevin Garnett
Chris Webber
Jamal Mashburn
Jason Kidd
Steve Nash
Ben Wallace
*********************************************************************************

In [49]:
X_train = torch.tensor(np.array(dataframe_00_10),dtype=torch.float) 
y_train = torch.tensor(mvp_list_train,dtype=torch.long).view(-1,1)

X_test = torch.tensor(np.array(dataframe_10_17),dtype=torch.float) 
y_test = torch.tensor(mvp_list_test,dtype=torch.long).view(-1,1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train.dtype)
print(y_train.dtype)
print(X_test.dtype)
print(y_test.dtype)

class Model(torch.nn.Module): 
    def __init__(self):
        super(Model, self).__init__() 
        self.linear1 = torch.nn.Linear(26, 10) 
        self.linear2 = torch.nn.Linear(10, 2) 
        self.sigmoid = torch.nn.Sigmoid()  #和上节的nn.function下的不一样
#         self.activate = torch.nn.Relu()
    def forward(self, x):
        x = self.sigmoid(self.linear1(x)) 
        x = self.sigmoid(self.linear2(x)) 
        return x

def train_model(model, X_train, y_train, X_test, y_test, epochs=250, batch_size=150, lr=0.01, weight_decay=0):
  train_dataset = TensorDataset(X_train, y_train)
  train_data_loader = DataLoader(train_dataset, batch_size=batch_size)
  # loss function
  loss_func = F.cross_entropy
  # optimizer
  optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
  # training loop
  for epoch in range(epochs):
    for xb, yb in train_data_loader:
      pred = model(xb)
      loss = loss_func(pred, yb)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
    y_pred = model(X_test)
    acc = accuracy_score(torch.argmax(y_pred, dim=1).detach().numpy(), y_test)
    print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))

    
model = Model()
train_model(model, X_train, y_train, X_test, y_test, lr=0.01)

torch.Size([1831, 26])
torch.Size([1831, 1])
torch.Size([1632, 26])
torch.Size([1632, 1])
torch.float32
torch.int64
torch.float32
torch.int64


RuntimeError: multi-target not supported at /tmp/pip-req-build-9oilk29k/aten/src/THNN/generic/ClassNLLCriterion.c:22

In [ ]:
# for epoch in range(30): 
#     # Forward
#     #注意这里没有minibatch，是整个batch
#     y_pred = model(X_train)
#     loss = criterion(y_pred, y_train)
#     print(epoch, loss.item())
#     optimizer.zero_grad()
#     loss.backward()    # Backward
#     optimizer.step()    # Update
    
#     pred = model(X_test)
#     acc = accuracy_score(torch.argmax(pred, dim=1).detach().numpy(), y_test)
#     print('Loss at epoch %d : %f, test acc: %f' % (epoch, loss, acc))
    
# y_pred = model(X_test)
# print("??")
# for i in range(1623):
#     print(i)
#     print(y_pred[i])



In [83]:
import torch
import torch.utils.data as Data

torch.manual_seed(1)   # 确保每次取的数据一致，使得实验结果一致

# 准备数据
x = torch.linspace(1,10,10)
y = torch.linspace(10,1,10)

# 转换成torch可以识别的Dataset
torch_dataset = Data.TensorDataset(x, y)    # 得到一个元组(x, y)

# 将dataset 放入DataLoader
loader = Data.DataLoader(
    dataset=torch_dataset,
    batch_size=5,
    shuffle=True,  # 每次训练打乱数据， 默认为False
    num_workers=0,  # 使用多进行程读取数据， 默认0，为不使用多进程
)

if __name__ == "__main__":
    # train
    for epoch in range(3):
        for step, (batch_x, batch_y) in enumerate(loader):
            print("Epoch: ", epoch, "| Step: ", step, "| batch x:",
                  batch_x.numpy(), "| batch y: ", batch_y.numpy())



Epoch:  0 | Step:  0 | batch x: [ 5.  7. 10.  3.  4.] | batch y:  [6. 4. 1. 8. 7.]
Epoch:  0 | Step:  1 | batch x: [2. 1. 8. 9. 6.] | batch y:  [ 9. 10.  3.  2.  5.]
Epoch:  1 | Step:  0 | batch x: [ 4.  6.  7. 10.  8.] | batch y:  [7. 5. 4. 1. 3.]
Epoch:  1 | Step:  1 | batch x: [5. 3. 2. 1. 9.] | batch y:  [ 6.  8.  9. 10.  2.]
Epoch:  2 | Step:  0 | batch x: [ 4.  2.  5.  6. 10.] | batch y:  [7. 9. 6. 5. 1.]
Epoch:  2 | Step:  1 | batch x: [3. 9. 1. 8. 7.] | batch y:  [ 8.  2. 10.  3.  4.]
